Function to fetch a number of images (1 to N) for a specific product, using Google Images.

In [ ]:
import requests
from bs4 import BeautifulSoup
from random import randint
from time import sleep

# Headers to look less suspicios when scraping
user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14'
headers = {'User-Agent': user_agent,'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

# Fetch a number {limit} of images from Google Images querying for {product}
def fetch_product(product, limit):
  # Sleep between 1 to 5 seconds to change the request pattern
  sleep(randint(1,5))
  # Google Images URL
  url = f"https://www.google.com/search?q={product}&tbm=isch"
  response = requests.get(url)
  # Parse the HTML content
  soup = BeautifulSoup(response.content, 'html.parser')
  #print(soup.prettify())
  image_tags = soup.find_all('img', attrs={'class': 'DS1iW'}) # Class tag changes frequently
  # Return the first {limit} images, if any
  if image_tags:
    result = []
    for i in range(min(limit, len(image_tags))):
      result.append(image_tags[i]['src'])
    return result
  else:
    return [""]

Functions to fill the data into the map that will generate the Dataframe

In [ ]:
# From a path, return the HTML tag of an image sourcing it
def path_to_image_html(path):
  print(path)
  return '<img src="' + path + '" width="100" >'

# Fill the various data in the map that will be used for the Dataframe
def fill_data(data, row, picture):
  data['Image'].append(picture)
  data['Season'].append(row['Season'])
  data['Brand'].append(row['Brand'])
  data['SKU'].append(row['SKU'])
  data['Description'].append(row['Description'])
  data['Style'].append(row['Style'])
  data['Colour Code'].append(row['Colour Code'])
  data['Colour Desc'].append(row['Colour Desc'])
  data['Size'].append(row['Size'])
  data['Gender'].append(row['Gender'])
  data['Category'].append(row['Category'])
  data['Sub-Category'].append(row['Sub-Category'])
  data['DDP-EUR'].append(row['DDP-EUR'])
  data['Alpi Servizio Moda'].append(row['Alpi Servizio Moda'])
  data['Alpi UK'].append(row['Alpi UK'])

A CSV log is used to save the images that were scraped in the past.

In [ ]:
from google.colab import drive
import csv

# Mount your Google Drive
drive.mount('/content/drive')

# Path to the CSV log file on Google Drive
log_file_path = '/content/drive/My Drive/Data/ATS_log.csv'

log_map = {}

# Fill the map containing SKU - Image path reading each line of the log
with open(log_file_path, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
      if row['SKU'] not in log_map:
        log_map[row['SKU']] = row['Image']

Process the ATS file and fetch images for each product

In [ ]:
from google.colab import drive
import csv
import pandas as pd
from IPython.core.display import display,HTML

# Mount your Google Drive
drive.mount('/content/drive')

# Path to the CSV file on Google Drive
csv_file_path = '/content/drive/My Drive/Data/ATS_cut.csv'

# Data to be print
data = {
    'Image': [],
    'Season': [],
    'Brand': [],
    'SKU': [],
    'Description': [],
    'Style': [],
    'Colour Code': [],
    'Colour Desc': [],
    'Size': [],
    'Gender': [],
    'Category': [],
    'Sub-Category': [],
    'DDP-EUR': [],
    'Alpi Servizio Moda': [],
    'Alpi UK': []
}

last_product_fetched = ""

# Open the CSV file and read each line
with open(csv_file_path, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
      # Only consider products in Italian warehouses
      if row['Alpi Servizio Moda'] or row['Alpi UK']:
        product = row['Brand'] + ' ' + row['Style'] # Text that will be queried
        print(product)
        # Only fetch new products, skip duplicate queries and data saved on the log
        if product != last_product_fetched:
          if row['SKU'] in log_map:
            print('Reading from log')
            picture = log_map[row['SKU']]
            fill_data(data, row, picture)
          else:
            pictures = fetch_product(product, 1)
            print('Fetched ' + pictures[0])
            last_product_fetched = product
            fill_data(data, row, path_to_image_html(pictures[0]))

# Create a DataFrame from the data dictionary
df = pd.DataFrame(data)
pd.set_option('display.max_colwidth', None)

# Save the DataFrame in HTML
df.to_html('ATS.html', escape=False)

# Save the DataFrame in CSV for logging purposes
df.to_csv(log_file_path, index=False)
